In [3]:
#import dependencies
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gmaps
import os 
import pprint
import urllib

#import yelp_key
from config import yelp_key 

In [4]:
cities = ['Philadelphia','Dallas','Indianapolis','Atlanta','Seattle']
restaurant_ethnicities_df = pd.read_csv("../Resources/rest_enthinicity.csv")
ethnicities = [ethnicity for ethnicity in restaurant_ethnicities_df['ethnicity'] ]
ethnicities[ethnicities.index('indian')] = 'indpak'
ethnicities[ethnicities.index('new_american')] = 'newamerican'
ethnicities

['chinese',
 'french',
 'greek',
 'indpak',
 'italian',
 'japanese',
 'brazilian',
 'mexican',
 'newamerican',
 'soulfood',
 'thai']

In [5]:
responses = []
API_HOST = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer %s' % yelp_key}

for city in cities:
    for ethnicity in ethnicities:     
        params={'term':'restaurants','location':f'{city}', 'categories':f'{ethnicity}', 'offset':0, 'limit' : 50}
        response = requests.get(API_HOST,headers=headers, params=params).json()
        response['ethnicity'] = f'{ethnicity}'
        response['city'] = f'{city}'
        responses.append(response)
        if response['total'] > 50:
            offset = 50
            while offset < response['total']:
                params={'term':'restaurants','location':f'{city}', 'categories':f'{ethnicity}', 'offset': offset,'limit' : 50}
                response = requests.get(API_HOST,headers=headers, params=params).json()
                response['ethnicity'] = f'{ethnicity}'
                response['city'] = f'{city}'
                responses.append(response)
                offset = offset + 50     

In [6]:
ethnicity_response_list = [responses[i]['ethnicity'] for i in range(len(responses))]
city_response_list = [responses[i]['city'] for i in range(len(responses))]

In [7]:
name = []
city = []
latitude = []
longitude = []
zipcode = []
ethnicity = []
price = []
rating = []

for i in range(len(responses)):
    for j in range(len(responses[i]['businesses'])):
        name.append(responses[i]["businesses"][j]["name"])
        city.append((responses[i]['city']))
        latitude.append(responses[i]["businesses"][j]["coordinates"]["latitude"])
        longitude.append(responses[i]["businesses"][j]["coordinates"]["longitude"])
        zipcode.append(responses[i]["businesses"][j]["location"]["zip_code"])
        ethnicity.append((responses[i]['ethnicity']))
        try:
            price.append(responses[i]["businesses"][j]["price"])
        except KeyError:
            price.append("N/A")
        rating.append(responses[i]["businesses"][j]["rating"])

In [8]:
rest_df = pd.DataFrame({
    "name": name,
    "city": city,
    "latitude": latitude,
    "longitude": longitude,
    "zipcode": zipcode,
    "ethnicity": ethnicity,
    "price": price,
    "rating": rating
})

rest_df

,name,city,latitude,longitude,zipcode,ethnicity,price,rating
0,Chubby Cattle,Philadelphia,39.954980,-75.156220,19107,chinese,N/A,4.5
1,Nan Zhou Hand Drawn Noodle House,Philadelphia,39.955401,-75.156901,19107,chinese,$,4.0
2,Dim Sum Garden,Philadelphia,39.955399,-75.156727,19107,chinese,$,4.0
3,Dim Sum House,Philadelphia,39.951885,-75.172466,19103,chinese,$$,4.0
4,Dan Dan,Philadelphia,39.950249,-75.167595,19102,chinese,$$,4.0
...,...,...,...,...,...,...,...,...
5739,Hanuman Thai Cafe,Seattle,47.676310,-122.207510,98033,thai,$$,3.5
5740,Jasmine Thai Cuisine,Seattle,47.609533,-122.341359,98101,thai,$,3.0
5741,Thai Ginger,Seattle,47.591233,-122.334065,98134,thai,$$,2.5
5742,Nibbana Thai Restaurant,Seattle,47.615154,-122.196783,98004,thai,$$,3.5


In [10]:
rest_df.to_csv("../Output/Data/yelp_data.csv", header = True, index = False)